In [1]:
import sys
import os
import numpy as np
import gc

import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
from datetime import datetime
import warnings
import shutil


In [8]:
def load_datasets(train_path, val_path, test_path):
  val_img_transform = transforms.Compose([transforms.Resize((244,244)),
                                          transforms.ToTensor()])
  train_img_transform = transforms.Compose([
#                                             transforms.RandomHorizontalFlip(), 
#                                             transforms.RandomAffine(degrees=0, scale=(0.8,1.2), shear=0.2, translate=(0.2, 0.2)), 
                                            transforms.AutoAugment(),
                                            transforms.Resize((244,244)),
                                            transforms.ToTensor()])
  train_dataset = datasets.ImageFolder(train_path, transform=train_img_transform)
  val_dataset = datasets.ImageFolder(val_path, transform=val_img_transform) 
  test_dataset = datasets.ImageFolder(test_path, transform=val_img_transform) if test_path is not None else None
  return train_dataset, val_dataset, test_dataset
    
def construct_dataloaders(train_set, val_set, test_set, batch_size, shuffle=True):
  train_dataloader = torch.utils.data.DataLoader(train_set, batch_size, shuffle)
  val_dataloader = torch.utils.data.DataLoader(val_set, batch_size) 
  test_dataloader = torch.utils.data.DataLoader(test_aset, batch_size) if test_path is not None else None
  return train_dataloader, val_dataloader, test_dataloader

In [3]:
@torch.no_grad()
def eval_model(data_loader, model, loss_fn, DEVICE):
  model.eval()
  loss, accuracy = 0.0, 0.0
  n = len(data_loader)

  for i, (x,y) in enumerate(data_loader):
    x,y = x.to(DEVICE), y.to(DEVICE)
    pred = model(x)
    loss += loss_fn(pred, y)/len(x)
    pred_label = torch.argmax(pred, axis = 1)
    accuracy += torch.sum(pred_label == y)/len(x)
    
  return loss/n, accuracy/n

In [9]:
def getVGGModel():
  vgg16 = models.vgg16_bn(weights=models.vgg.VGG16_BN_Weights.IMAGENET1K_V1)

  # Fix the conv layers parameters
  for conv_param in vgg16.features.parameters():
    conv_param.require_grad = False

  classifications = nn.Sequential(
    nn.Flatten(),
    nn.Linear(25088,1024),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(1024,3)
  )

  vgg16.classifier = classifications

  return vgg16

In [ ]:
def load_checkpoint(checkpoint_path):
  checkpoint = torch.load(checkpoint_path, map_location=torch.device('cuda'))
  return checkpoint

In [ ]:
def load_model_fm_checkpoint(checkpoint, primitive_model):
  primitive_model.load_state_dict(checkpoint['model_state_dict'])
  return primitive_model

In [10]:
checkpoint_path = "/home1/09308/zhengmk/ML-Training-2022/output_model/best_model.pt"
checkpoint = load_checkpoint(checkpoint_path, map_location=torch.device('cuda'))

print(f"Best loss: {checkpoint['loss']}, best accuracy: {checkpoint['accuracy']}")

Best loss: 0.04212700575590134, best accuracy: 0.8187500238418579


In [11]:
DEVICE = 'cuda'
batch_size = 80
train_path, val_path, test_path = "/tmp/Dataset_2/Train/", "/tmp/Dataset_2/Validation/", None

loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1).cuda()
train_set, val_set, _ = load_datasets(train_path, val_path, test_path)
_, val_loader, _ = construct_dataloaders(train_set, val_set, None, batch_size, True)
model = getVGGModel().to(DEVICE)
model = load_model_fm_checkpoint(checkpoint, model)

<All keys matched successfully>

In [12]:
loss, acc = eval_model(val_loader, model, loss_fn, DEVICE)
print(f"Val loss: {loss}, Val accuracy: {acc}") 

Val loss: 0.00016985634283628315, Val accuracy: 0.6831955909729004
